In [ ]:
import numpy as np
import matplotlib.pylab as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
import sys
import matplotlib as mpl
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as pl
from mpmath import *
#import plotly.graph_objects as go
from typing import Tuple, Iterable
#from plotly.subplots import make_subplots
#import plotly.express as px
import pandas as pd
import scipy.special as sp_spec

In [ ]:
gizi=0.04 
giz=0.01
Eiz=-65.
Eizi=-60.
Ee=0.0
Ei=-80.0
Qi=5.0
Qe=1.5
tauize=1
tauizi=1
tau_adap=1
a_adap=1
d_adap=15
b_adap=0
tause=5e-3
tausi=5e-3
Ke=8000*0.05
Ki=2000*0.05
ue=0
T=0.015

In [ ]:
def sv(ne,ni,ue):
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    ui=0
    #ue=0
    ue=ue
    muve=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    muvi=((2*gizi*Eizi+muge+mugi)-np.sqrt((2*gizi*Eizi+muge+mugi)**2-4*gizi*(gizi*Eizi**2+muge*Ee+mugi*Ei-ui)))/(2*gizi)

In [ ]:
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
 
    be=Qe*(Ee-muve)#/mug  Note that we are not using mug!!!
    bi=Qi*(Ei-muvi)#/mug
    ce=tause
    ci=tausi
    
    argsv=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)   +bi**2*ci**3/(8*tauizi**2))
    if argsv>0:
    	sv=np.sqrt(Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2)))
    else:
    	sv=1e-9
    
    #print('sv=',sv)
    return sv
    
def tauv(ne,ni,ue):
    
    ne=ne+1e-9
    ni=ni+1e-9
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    ui=0
    #ue=ue
    muve=((2*giz*Eiz+muge+mugi+b_adap)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    muvi=((2*gizi*Eizi+muge+mugi)-np.sqrt((2*gizi*Eizi+muge+mugi)**2-4*gizi*(gizi*Eizi**2+muge*Ee+mugi*Ei-ui)))/(2*gizi)
    
    ae=giz*(muve-Eiz)**2
    ai=gizi*(muvi-Eizi)**2
    be=Qe*(Ee-muve)
    bi=Qi*(Ei-muvi)
    ce=tause
    ci=tausi
  
    
    argv=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2))
    if argv >0:
    	svv2=Ke*ne*(2*ae*be*ce**3/tauize**2+ce**3*be**2/(8*tauize)+be**2*ce**3/(8*tauize**2)) + Ki*ni*(2*ai*bi*ci**3/tauizi**2+ci**3*bi**2/(8*tauizi)+bi**2*ci**3/(8*tauizi**2))
    
    	tauv=0.5*(Ke*ne*(be**2*ce**4/(2*np.pi*tauize**2))+Ki*ni*(bi**2*ci**4/(2*np.pi*tauizi**2)))/(svv2+1e-9)
        
    else:
      tauv=1
      #print('tauv=',tauv)
    return tauv

In [ ]:
def TF(nu,P,ue,giz,Eiz):
    ne,ni=nu
    Po,Pmuv,Psv,Ptauv,Pvsv,Pvtauv,Psvtauv,Pvv,Ptt,Pss=P
    #ne=nu
    #ni=8
    muge=Qe*ne*tause*Ke
    mugi=Qi*ni*tause*Ki
    mug=muge+mugi+giz
    ui=0
    #ue=ue
    muvo=-45
    dmuvo=2
    svo=4
    dsvo=5
    tauvo=0.005
    dtauvo=0.05
    
    
    muv=((2*giz*Eiz+muge+mugi)-np.sqrt((2*giz*Eiz+muge+mugi+b_adap)**2-4*giz*(giz*Eiz**2+muge*Ee+mugi*Ei-ue)))/(2*giz)
    #muv=muve

In [ ]:
 
    Pscale=1.
    noutf=Pscale*(sp_spec.erfc(((Po + Pmuv*(muv-muvo)/dmuvo + Psv*(sv(ne,ni,ue)-svo)/dsvo + Ptauv*(tauv(ne,ni,ue)-tauvo)/dtauvo \
            +Pvsv*(muv-muvo)*(sv(ne,ni,ue)-svo)/(dsvo*dmuvo) + Pvtauv*(muv-muvo)*(tauv(ne,ni,ue)-tauvo)/(dtauvo*dmuvo)\
            +Psvtauv*(sv(ne,ni,ue)-svo)*(tauv(ne,ni,ue)-tauvo)/(dtauvo*dsvo) + Pvv*(muv-muvo)**2/(dmuvo*dmuvo) \
            +Ptt*(tauv(ne,ni,ue)-tauvo)**2/(dtauvo*dtauvo) + Pss*(sv(ne,ni,ue)-svo)**2/(dsvo*dsvo))-muv)/(np.sqrt(2)*sv(ne,ni,ue)) ))/(2*tauv(ne,ni,ue))
 
        
    return noutf

In [ ]:
external_input=10

In [ ]:
f = open('RS_fit.txt', 'r')
lines = f.readlines()
PRS=np.zeros(10)

In [ ]:
for i in range(0,len(PRS)):
    PRS[i]=lines[i]

In [ ]:
f.close()

In [ ]:
f = open('FS_fit.txt', 'r')
lines = f.readlines()
PFS=np.zeros(10)

In [ ]:
for i in range(0,len(PRS)):
    PFS[i]=lines[i]

In [ ]:
f.close()

RS=np.asarray(PRS)<br>
FS=np.asarray(PFS)<br>
rint PRS

In [ ]:
tfinal=2.
dt=0.0001

In [ ]:
t = np.linspace(0, tfinal, int(tfinal/dt))

In [ ]:
f = open('time.txt', 'wb')

In [ ]:
fecont=3;
ficont=7;
dRS=15
aRS=1
w=0.5*fecont*dRS/aRS

In [ ]:
LSw=[]
LSfe=[]
LSfi=[]

In [ ]:
for i in range(len(t)):
    
    fecontold=fecont
    fecont+=dt/T*(TF((fecont+external_input,ficont), PRS,w,giz,Eiz)-fecont) 
    w+=dt*( -aRS*w+(dRS)*fecontold)
    ficont+=dt/T*(TF((fecont+external_input,ficont),PFS,0,gizi,Eizi) - ficont) 
    LSfe.append(float(fecont))
    LSfi.append(float(ficont))
    LSw.append(float(w))
    

In [ ]:
plt.plot(t, LSfe)
plt.plot(t, LSfi)
fig=plt.figure()
#plt.plot(LSfe, LSfi)
ax = fig.add_subplot(1, 1, 1, projection = '3d')
ax.plot(LSfe, LSfi, LSw)
plt.figure()
plt.plot(LSfe,LSfi)
plt.figure()
plt.plot(t, LSw)
plt.show()

.close()